[comment]: # (Attach Default Lakehouse Markdown Cell)
# 📌 Attach Default Lakehouse
❗**Note the code in the cell that follows is required to programatically attach the lakehouse and enable the running of spark.sql(). If this cell fails simply restart your session as this cell MUST be the first command executed on session start.**

In [ ]:
%%configure
{
    "defaultLakehouse": {  
        "name": "{{lakehouse_name}}",
    }
}

# 📦 Pip
Pip installs reqired specifically for this template should occur here

In [ ]:
# No pip installs needed for this notebook

# 🔗 Imports

In [ ]:
from notebookutils import mssparkutils # type: ignore
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.functions import lower, udf, col, row_number, when, monotonically_increasing_id
from pyspark.sql.window import Window
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
import json
import hashlib
import yaml
import re


# { } Params

In [53]:
# Set Lakehouse
RelativePathForMetaData = "Files/metaextracts/"

StatementMeta(, 79594e22-45e2-4587-a343-c5b3f00bc4fb, 55, Finished, Available)

# #️⃣ Functions

In [54]:
def execute_sql_and_write_to_file(sql_query, file_path):
    # Execute the SQL query
    df = spark.sql(sql_query) # type: ignore
    df_to_file(df, file_path)

def df_to_file(df, file_path):

    # Convert the DataFrame to a Pandas DataFrame
    pandas_df = df.toPandas()

    # Convert the Pandas DataFrame to a string
    df_string = pandas_df.to_json(None, orient='records')

    # Write the string to the file
    mssparkutils.fs.put(file_path, df_string, True) # type: ignore


StatementMeta(, 79594e22-45e2-4587-a343-c5b3f00bc4fb, 56, Finished, Available)

# List Schemas Macro Export

In [55]:
#convert database/ lakehouse name to lower case
#execute_sql_and_write_to_file("show databases", f"{RelativePathForMetaData}/ListSchemas.json")
df_database = spark.sql("show databases")
df_database = df_database.withColumn('namespace', lower(df_database['namespace']))

df_to_file (df_database, f"{RelativePathForMetaData}/ListSchemas.json")

StatementMeta(, 79594e22-45e2-4587-a343-c5b3f00bc4fb, 57, Finished, Available)

# List Relations

In [56]:
# Define the schema
schema = StructType([
    StructField("namespace", StringType(), True),
    StructField("tableName", StringType(), True),
    StructField("isTemporary", BooleanType(), True),
    StructField("information", StringType(), True),
    StructField("type", StringType(), True)
])

# Create an empty DataFrame with the schema
union_df = spark.createDataFrame([], schema) # type: ignore


# Define a UDF to extract the Type from metadata
def extract_type(metadata):
    match = re.search(r"Type:\s*(\w+)", metadata)
    if match:
        return match.group(1)
    return None

extract_type_udf = udf(extract_type, StringType())

# Function to execute SQL operations
def execute_sql_operations(namespace):
    sql = f"show table extended in {namespace} like '*'"
    df_temp = spark.sql(sql)  # type: ignore
    df_temp = df_temp.withColumn("type", extract_type_udf(df_temp["information"]))
    
    return df_temp


# Run SQL operations in parallel using ThreadPoolExecutor
def run_sql_operations_in_parallel(namespaces):
    results = []
    with ThreadPoolExecutor() as executor:
        future_to_namespace = {executor.submit(execute_sql_operations, row['namespace']): row['namespace'] for row in namespaces}
        for future in as_completed(future_to_namespace):
            namespace = future_to_namespace[future]
            try:
                df_temp = future.result()
                results.append(df_temp)
            except Exception as exc:
                print(f'{namespace} generated an exception: {exc}')
    return results

df = spark.sql("show databases") # type: ignore
namespaces = df.collect()

# Run SQL operations in parallel and collect results
results = run_sql_operations_in_parallel(namespaces)

# Union all DataFrames
for df_temp in results:
    union_df = union_df.union(df_temp)

# Lowercasing removed as it causes issues with describe statements eg. describe unable to find objects. ## John Rampono: 11-Sep-24
#convert database/ lakehouse name and table name to lower case
# union_df = union_df.withColumn('namespace', lower(union_df['namespace']))
# union_df = union_df.withColumn('tableName', lower(union_df['tableName']))

# Remove noise 
union_df = union_df.filter((col("namespace") != "") & (col("namespace").isNotNull()))

df_to_file(union_df, f"{RelativePathForMetaData}/ListRelations.json")

StatementMeta(, 79594e22-45e2-4587-a343-c5b3f00bc4fb, 58, Finished, Available)

# Describe Table Extended

In [60]:
import json
from pyspark.sql.types import StructType, StructField, StringType, BooleanType # type: ignore
from pyspark.sql.functions import lit # type: ignore


# Define the schema
schema = StructType([
    StructField("col_name", StringType(), True),
    StructField("data_type", StringType(), True),
    StructField("comment", StringType(), True), 
    StructField("namespace", StringType(), True),
    StructField("tableName", StringType(), True),
])


# Create an empty DataFrame with the schema
union_df = spark.createDataFrame([], schema) # type: ignore
union_dtable = spark.createDataFrame([], schema) # type: ignore



# Define a function to execute the SQL operations
def execute_sql_operations(j):
    # Use statement swapped for two part naming as it did not appear to be working ## John Rampono: 11-Sep-24
    # sql = f"use {j['namespace']}"
    # spark.sql(sql)  # type: ignore
    sql = f"describe extended {j['namespace']}.{j['tableName']}"
    df_temp = spark.sql(sql)  # type: ignore
        
    # Add a row number column using monotonically_increasing_id
    df_with_row_num = df_temp.withColumn("row_num", monotonically_increasing_id())

    # Identify the first row where col_name is null or empty
    first_null_or_empty_row = df_with_row_num.filter((col("col_name") == "") | col("col_name").isNull()).orderBy("row_num").limit(1)

    # Get the row number of the first null or empty row
    first_null_or_empty_row_num = first_null_or_empty_row.select("row_num").collect()

    if first_null_or_empty_row_num:
        first_null_or_empty_row_num = first_null_or_empty_row_num[0]["row_num"]
        # Filter the DataFrame to keep only rows before this row
        df_filtered = df_with_row_num.filter(col("row_num") < first_null_or_empty_row_num).drop("row_num")
    else:
        # If no null or empty row is found, keep the original DataFrame
        df_filtered = df_temp    
    
    df_filtered = df_filtered.withColumn('namespace', lit(j['namespace']))
    df_filtered = df_filtered.withColumn('tableName', lit(j['tableName']))

    # generate Schema.yml
    sqldesc = f"describe {j['namespace']}.{j['tableName']}"
    df_dtable = spark.sql(sqldesc)  # type: ignore
    
    # Lowercasing removed as it causes issues with describe statements eg. describe unable to find objects. ## John Rampono: 11-Sep-24
    df_dtable = df_dtable.withColumn('namespace', lit(j['namespace']))
    df_dtable = df_dtable.withColumn('tableName', lit(j['tableName']))

    return df_filtered, df_dtable

# Run SQL operations in parallel using ThreadPoolExecutor
def run_sql_operations_in_parallel(jo):
    union_df = None
    union_dtable = None

    with ThreadPoolExecutor() as executor:
        future_to_sql = {executor.submit(execute_sql_operations, j): j for j in jo}
        for future in as_completed(future_to_sql):
            j = future_to_sql[future]
            try:
                df_temp, df_dtable = future.result()
                if union_df is None:
                    union_df = df_temp
                else:
                    union_df = union_df.union(df_temp)

                if union_dtable is None:
                    union_dtable = df_dtable
                else:
                    union_dtable = union_dtable.union(df_dtable)
            except Exception as exc:
                tableName = j['tableName']
                namespace = j['namespace']
                print(f'{namespace}.{tableName} generated an exception: {exc}')

    return union_df, union_dtable


data = spark.sparkContext.wholeTextFiles(f"{RelativePathForMetaData}/ListRelations.json").collect() # type: ignore
file_content = data[0][1]
jo = json.loads(file_content)

# Filter out views as these will cause issues  ## John Rampono: 11-Sep-24
filtered_jo = [item for item in jo if item["type"].lower() == "managed"]

union_df, union_dtable = run_sql_operations_in_parallel(filtered_jo)

# Lowercasing removed as it causes issues with describe statements eg. describe unable to find objects. ## John Rampono: 11-Sep-24
#convert column name to lower case
#union_df = union_df.withColumn('col_name', lower(union_df['col_name']))
#union_dtable = union_dtable.withColumn('col_name', lower(union_dtable['col_name']))

# Do we still need this view?
union_dtable.createOrReplaceTempView("describetable")

#create file
df_to_file(union_df, f"{RelativePathForMetaData}/DescribeRelations.json")

StatementMeta(, 79594e22-45e2-4587-a343-c5b3f00bc4fb, 62, Finished, Available)

# Schema yaml generation

In [ ]:
tables = {}
for item in union_dtable.collect():
    #print(item)
    table_name = item['tableName']
    col_name = item['col_name']
    
    if table_name not in tables:
        tables[table_name] = []
    
    if col_name:  # Only add columns with a name
        tables[table_name].append({
            "name": col_name,
            "description": "add column description"
        })

output_data = {}
output_data["version"] = 2
models = []
for table_name, columns in tables.items():
    models.append({
        "name": table_name,
        "description": "add table description",
        "columns": columns
    })
output_data["models"] = models

yaml_string = yaml.dump(output_data,  default_flow_style=False)
RelativePathForShemaTemplate = "Files/SchemaTemplate"
file_path = f"{RelativePathForShemaTemplate}/schema.yml"
mssparkutils.fs.put(file_path, yaml_string, True) # type: ignore

# #️⃣ Create Hash for Files 

In [ ]:
# Define the schema
schema = StructType([
    StructField("hash", StringType(), True),
    StructField("file", StringType(), True)
])

files = ["ListSchemas","DescribeRelations","ListRelations"]

df = spark.createDataFrame([], schema) # type: ignore
for file in files:
    data = spark.sparkContext.wholeTextFiles(f"{RelativePathForMetaData}/{file}.json").collect() # type: ignore
    file_content = data[0][1]
    hashinfo = {}
    hashinfo['hash'] = hashlib.sha256(file_content.encode('utf-8')).hexdigest()
    hashinfo['file'] = f"{file}.json"
    df = df.union(spark.createDataFrame([hashinfo], schema)) # type: ignore
    
df_to_file(df, f"{RelativePathForMetaData}/MetaHashes.json") # type: ignore
